## Predictive modeling with Tweedie distributed response  
*Gorkem Ozkaya*

In this notebook we generate data according to the linear Tweedie model with a log link. Then we compare three predictive models: 
* Tweedie GLM with the `cplm` package
* Gradient boosting with the `xgboost` package, using the `reg:tweedie` objective, where the loss is likelihood of the mean for Tweedie distribution
* Gradient boosting with the `xgboost` package, using the `reg:linear` objective, where the loss is simple least squares

In [1]:
library(tweedie)
library(cplm)
library(xgboost)

Warning message:
: package ‘cplm’ was built under R version 3.3.2Loading required package: coda
Warning message:
: package ‘coda’ was built under R version 3.3.2Loading required package: Matrix
Loading required package: splines


Function that generates data according to the linear model: 

In [2]:
gen_data <- function (N = 10000, p = 1.5, phi=100) {
  var1 <- rnorm(N)
  var2 <- rnorm(N)
  var3 <- rnorm(N)
  y <- exp(7 + 0.1*var1 + 0.2*var2 - 0.3*var3)
  
  resp = rep(0, N)
  
  for (i in 1:N) {
    resp[i] <- rtweedie(1, xi = p, mu = y[i], phi=phi )
  }
  
  return(data.frame(var1, var2, var3, mu=y, resp))
}

### Generating training and test sets

In [3]:
dt <- gen_data()
dt_test <- gen_data()

# for xgboost models
d_train <- xgb.DMatrix(data = as.matrix(dt[, c("var1", "var2", "var3")]), label = dt$resp, missing = NA)
d_test <- xgb.DMatrix(data = as.matrix(dt_test[, c("var1", "var2", "var3")]), label = dt_test$resp, missing = NA)

### Tweedie GLM

In [4]:
model_glm <- cpglm(resp ~ var1+var2+var3, data=dt, optimizer="bobyqa")

y_hat_glm <- predict(model_glm, dt_test[, c("var1", "var2", "var3")])

### XGBoost with Tweedie objective

In [5]:
params_tweedie <- list(
  objective = 'reg:tweedie',
  eval_metric = 'rmse', 
  tweedie_variance_power = 1.5,
  max_depth = 6,
  eta = 0.01)


bst_tweedie <- xgb.train(
  data = d_train, 
  params = params_tweedie, 
  maximize = FALSE,
  watchlist = list(train = d_train, test=d_test), 
  nrounds = 1000,
  print_every_n = 50)


y_hat_xgb_tweedie <- predict(bst_tweedie, d_test)

[1]	train-rmse:2476.955811	test-rmse:2351.621582 
[51]	train-rmse:2475.586670	test-rmse:2350.224854 
[101]	train-rmse:2471.941650	test-rmse:2346.528320 
[151]	train-rmse:2462.552734	test-rmse:2337.097168 
[201]	train-rmse:2440.162842	test-rmse:2314.962891 
[251]	train-rmse:2394.323730	test-rmse:2271.380859 
[301]	train-rmse:2320.444580	test-rmse:2206.259766 
[351]	train-rmse:2230.586426	test-rmse:2137.055908 
[401]	train-rmse:2145.951416	test-rmse:2083.771240 
[451]	train-rmse:2079.055176	test-rmse:2053.506104 
[501]	train-rmse:2030.420288	test-rmse:2039.423462 
[551]	train-rmse:1997.380737	test-rmse:2033.699219 
[601]	train-rmse:1974.729614	test-rmse:2031.606323 
[651]	train-rmse:1958.262329	test-rmse:2031.567017 
[701]	train-rmse:1945.983887	test-rmse:2031.985962 
[751]	train-rmse:1935.498047	test-rmse:2032.644165 
[801]	train-rmse:1926.630981	test-rmse:2033.466919 
[851]	train-rmse:1918.441528	test-rmse:2034.266846 
[901]	train-rmse:1910.293213	test-rmse:2035.266479 
[951]	train-rms

### XGBoost with Least squares objective

In [6]:
params_leastsq <- list(
  objective = 'reg:linear',
  eval_metric = 'rmse', 
  tweedie_variance_power = 1.5,
  max_depth = 6,
  eta = 0.01)


bst_leastsq <- xgb.train(
  data = d_train, 
  params = params_leastsq, 
  maximize = FALSE,
  watchlist = list(train = d_train, test=d_test), 
  nrounds = 1000,
  print_every_n = 50)


y_hat_xgb_leastsq <- predict(bst_leastsq, d_test)

[1]	train-rmse:2469.842773	test-rmse:2345.668457 
[51]	train-rmse:2204.150879	test-rmse:2137.756592 
[101]	train-rmse:2072.081299	test-rmse:2061.310059 
[151]	train-rmse:2003.094116	test-rmse:2037.714478 
[201]	train-rmse:1962.820557	test-rmse:2031.466309 
[251]	train-rmse:1935.417114	test-rmse:2031.223633 
[301]	train-rmse:1917.244751	test-rmse:2032.850098 
[351]	train-rmse:1901.709961	test-rmse:2035.406616 
[401]	train-rmse:1887.165039	test-rmse:2037.472778 
[451]	train-rmse:1877.370850	test-rmse:2039.023193 
[501]	train-rmse:1869.997681	test-rmse:2040.524902 
[551]	train-rmse:1861.443359	test-rmse:2042.084961 
[601]	train-rmse:1853.184937	test-rmse:2043.426636 
[651]	train-rmse:1843.502197	test-rmse:2045.074707 
[701]	train-rmse:1834.347168	test-rmse:2046.445557 
[751]	train-rmse:1824.339111	test-rmse:2047.975708 
[801]	train-rmse:1814.281372	test-rmse:2049.369385 
[851]	train-rmse:1803.211304	test-rmse:2050.776367 
[901]	train-rmse:1793.056763	test-rmse:2052.099609 
[951]	train-rms

### Comparing the results of the three models
Now we compare the model performances on the test sets using Gini indices, which is one of the standard performance measures in insurance:

In [7]:
df_gini <- data.frame(y = dt_test$resp, glm = y_hat_glm, xgb_tweedie = y_hat_xgb_tweedie, xgb_leastsq = y_hat_xgb_leastsq)
df_gini$base = mean(df_gini$y)
gini(loss = "y" , score = c("glm", "xgb_tweedie", "xgb_leastsq"), base = NULL, data=df_gini)


Call:
gini(loss = "y", score = c("glm", "xgb_tweedie", "xgb_leastsq"), 
    base = NULL, data = df_gini)

Gini indices:
             glm     xgb_tweedie  xgb_leastsq
glm           0.000   2.608        2.151     
xgb_tweedie  13.442   0.000        7.222     
xgb_leastsq  12.562   6.847        0.000     

Standard errors:
             glm     xgb_tweedie  xgb_leastsq
glm          0.0000  1.0031       0.9902     
xgb_tweedie  1.0469  0.0000       1.0563     
xgb_leastsq  0.9947  1.0205       0.0000     

The selected score is glm.

### Discussion
Looking at the Gini indices, we see that GLM performs the best.  This is expected, because we generated the data completely in accordance with the GLM Tweedie model assumptions.  

On the other hand, when we compare the XGBoost performances, we see no significant difference between using  *least squares* versus *Tweedie* objectives. 